### Upload a CAD File

This notebook demos how to upload a CAD file to a Spacecraft Branch.

#### Setup

Imports and notebook variables:

In [ ]:
from pathlib import Path
from urllib3.filepost import encode_multipart_formdata
from sedaro import SedaroApiClient

BRANCH_ID = ... # str
CAD_FILE = Path('path/to/cadfile.glb')
SCALE_FACTOR = ... # int, adjust as needed

#### Important: Read Before Running

This notebook makes changes to an agent branch indicated in the settings section. Ensure any changes to the target branch are saved prior to running this code. Sedaro recommends committing current changes and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
  "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.


In [ ]:
import json

with open('../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('../config.json', 'r') as file:
    config = json.load(file)

#### Upload the file

Upload the CAD file, replacing the existing file on the spacecraft.

In [ ]:
sedaro = SedaroApiClient(api_key=API_KEY, host=config['HOST'])

with sedaro.api_client() as api_client:
    with open(CAD_FILE, 'rb') as f:
        file_content = f.read()

        fields = (
            ('cadFile', (CAD_FILE.name, file_content, 'application/octet-stream')),
            ('cadScaleFactor', SCALE_FACTOR),
        )

        body, content_type = encode_multipart_formdata(fields)
        headers = {'Content-Type': content_type}

        api_client.call_api(
            f'/models/branches/{BRANCH_ID}/cad/',
            'POST',
            headers=headers,
            body=body,
        )